# Redes Neurais Artificiais

### Descriçao

---
1. O objetivo deste notebook é a construir sua primeira rede neural.
2. Utilizaremos o conjunto de dados proposto no [gist](https://gist.githubusercontent.com/batestin1/e09def5cbd3732e345029f5c88155182/raw/a14ff6964c3fbeeca3b7f17ec32102f0ade14828/gistfile1.txt)

3. O problema consiste em prever e classificar a perda de clientes que a empresa esta sofrendo. Investigar os motivos para isso.
---

# Instalação dos pacotes

In [1]:
!pip install pandas numpy scikit-learn keras

# Importando as Bibliotecas

In [2]:
import os  # Para manipulação de pastas e diretórios (ex: listar arquivos, criar pastas, etc)
import pandas as pd  # Para manipulação de dados em formato de tabelas (DataFrames)
import numpy as np  # Para criação e manipulação de matrizes e operações matemáticas

# Importa módulos do scikit-learn para pré-processamento e avaliação de modelos
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler  # Codificação de variáveis categóricas e normalização de dados
from sklearn.compose import make_column_transformer  # Para aplicar transformações específicas em colunas do dataset
from sklearn.model_selection import train_test_split  # Para dividir o dataset em conjuntos de treino e teste
from sklearn.metrics import confusion_matrix  # Para avaliar a performance do modelo com a matriz de confusão

# Importa bibliotecas para criação de redes neurais
import tensorflow as tf  # Biblioteca principal para criação e treinamento de modelos de machine learning (ML) e deep learning (DL)

from keras.models import Sequential  # Modelo linear (camada a camada) para construir redes neurais de forma sequencial
from keras.layers import Dense  # Camadas densamente conectadas (fully connected layers) para redes neurais

from keras.models import load_model  # Para carregar modelos treinados salvos anteriormente

# Obtendo o dataset

In [3]:
# Lê um arquivo CSV diretamente de uma URL e armazena em um DataFrame
df = pd.read_csv('https://gist.githubusercontent.com/batestin1/e09def5cbd3732e345029f5c88155182/raw/a14ff6964c3fbeeca3b7f17ec32102f0ade14828/gistfile1.txt')

# Exibe as primeiras 5 linhas do DataFrame para visualização inicial dos dados
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Conhecendo o dataset

In [4]:
# Exibe informações gerais sobre o DataFrame,
# como o número de entradas (linhas), número de colunas, tipos de dados e valores não nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


##### Pre Processamento dos dados

In [5]:
# Separando os dados em variáveis independentes (X) e dependente (y)

# 'X' recebe as colunas de 3 até 12 (lembrando que o iloc é exclusivo no final, então a 13ª coluna não entra)
# Essas serão as features (variáveis preditoras) que o modelo usará para aprender
X = df.iloc[:, 3:13].values

# 'y' recebe apenas a coluna 13, que será o alvo (variável dependente) que queremos prever
y = df.iloc[:, 13].values

In [6]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [7]:
y

array([1, 0, 1, ..., 1, 1, 0])

In [10]:
# Transformação de dados categóricos

# Cria um LabelEncoder para transformar a coluna de países (segunda coluna de X)
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])  # Codifica os países como números inteiros

# Cria outro LabelEncoder para transformar a coluna de gênero (terceira coluna de X)
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])  # Codifica o gênero como números inteiros

# Aplica OneHotEncoder para a coluna de países (agora numérica) para criar variáveis binárias (0 ou 1)
# 'remainder="passthrough"' indica que o restante das colunas será mantido sem alteração
ohe = make_column_transformer(
  (OneHotEncoder(categories='auto', handle_unknown='ignore'), [1]),
  remainder="passthrough"
)

# Ajusta o OneHotEncoder e transforma X
X = ohe.fit_transform(X)

# Remove a primeira coluna criada pelo OneHotEncoder para evitar o problema de multicolinearidade (dummy variable trap)
X = X[:, 1:]

#### Dividir dados em treino e teste

In [11]:
# Dividindo o conjunto de dados em treino e teste

# 'train_test_split' separa os dados:
# 80% dos dados para treino e 20% para teste (test_size=0.2)
# 'random_state=42' garante que a divisão seja sempre a mesma ao rodar o código (reprodutibilidade)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### padronizando os dados

In [12]:
# Padronizando os dados (feature scaling)

# Cria um StandardScaler para normalizar os dados
# O StandardScaler transforma os dados para que tenham média 0 e desvio padrão 1 (não necessariamente entre 0 e 1)
sc = StandardScaler()

# Ajusta o scaler aos dados de treino e transforma eles
X_train = sc.fit_transform(X_train)

# Apenas transforma os dados de teste usando o mesmo scaler (sem ajustar novamente)
X_test = sc.transform(X_test)

#### Perceptron (opcional)

In [13]:
# Criando uma cópia dos dados de treino e teste

# Copiamos os conjuntos para usar especificamente no perceptron,
# assim evitamos qualquer modificação acidental nos dados originais
X_train_p = X_train
X_Test_p = X_test
y_train_p = y_train
y_test_p = y_test

# Definindo nosso perceptron (rede neural mais simples possível)

# Criamos um modelo Sequential
# Adicionamos apenas uma camada densa (totalmente conectada) com:
# - 1 neurônio (saída binária)
# - função de ativação sigmoid (para problemas de classificação binária)
model = tf.keras.Sequential([
  tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilando o modelo

# - Otimizador 'adam' para atualização dos pesos
# - Função de perda 'binary_crossentropy' para classificação binária
# - Métrica de avaliação 'accuracy' (acurácia)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinando o modelo

# Ajustamos o modelo aos dados de treino
# - epochs=10: vamos passar 10 vezes pelo conjunto de dados
# - verbose=1: mostra o progresso do treinamento
model.fit(X_train_p, y_train_p, epochs=10, verbose=1)

# Avaliando o modelo

# Avaliamos o desempenho do modelo com os dados de teste
loss, accuracy = model.evaluate(X_Test_p, y_test_p)

# Exibimos a acurácia formatada em porcentagem
print(f'Acurácia do modelo: {round(accuracy, 2) * 100}%')

# Exibe um resumo da estrutura do modelo
model.summary()

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5620 - loss: 0.7361
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6628 - loss: 0.6229
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7466 - loss: 0.5495
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7784 - loss: 0.5132
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7986 - loss: 0.4798
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7947 - loss: 0.4707
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7997 - loss: 0.4572
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8029 - loss: 0.4476
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8059 - loss: 0.4444
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8136 - loss: 0.4316
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8010 - loss: 0.4302
Acurácia do modelo: 81.0%


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (32, 1)                     │              12 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38 (156.00 B)

 Trainable params: 12 (48.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 26 (108.00 B)

# Criando nossa RNA

In [14]:
# Inicializamos o modelo do tipo Sequential (camadas empilhadas linearmente)
classifier = Sequential()

# Adicionando camadas à rede

# Primeira camada oculta
# - units=6: número de neurônios na camada
# - kernel_initializer='uniform': inicializa os pesos de forma uniforme
# - activation='relu': função de ativação ReLU (retifica valores negativos para zero)
# - input_dim=X_train.shape[1]: número de entradas (atributos)
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=X_train.shape[1]))

# Segunda camada oculta
# - Também com 6 neurônios
# - Mesmas configurações de inicialização e ativação (ReLU)
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

# Camada de saída
# - units=1: apenas 1 neurônio para saída binária
# - activation='sigmoid': transforma o valor de saída entre 0 e 1 (interpretação como probabilidade)
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

# Compilando o modelo

# - optimizer='adam': algoritmo de otimização adaptativo (baseado em Gradiente Descendente)
# - loss='binary_crossentropy': função de perda usada para problemas de classificação binária
# - metrics=['accuracy']: avaliamos o modelo usando a métrica de acurácia
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinando o modelo

# - batch_size=10: número de amostras que serão propagadas antes de atualizar os pesos
# - epochs=30: número total de vezes que o modelo verá todo o conjunto de dados
classifier.fit(X_train, y_train, batch_size=10, epochs=30)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7865 - loss: 0.5667
Epoch 2/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7915 - loss: 0.4355
Epoch 3/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7921 - loss: 0.4280
Epoch 4/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8149 - loss: 0.4360
Epoch 5/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8284 - loss: 0.4117
Epoch 6/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8335 - loss: 0.4187
Epoch 7/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8321 - loss: 0.4114
Epoch 8/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8364 - loss: 0.4058
Epoch 9/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8310 - loss: 0.4148
Epoch 10/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8312 - loss: 0.4159
Epoch 11/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8326 - loss: 0.4085
Epoch 12/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

#### Prevendo dados de teste

In [15]:
# Fazendo a previsão (valores entre 0 e 1)
y_pred = classifier.predict(X_test)

# Visualizando a previsão em porcentagem (antes de aplicar o threshold de 0.5)
z_pred = np.array([f"{value[0] * 100:.2f}%" for value in y_pred])
print(z_pred)

# Depois você pode aplicar o threshold para transformar em 0 ou 1
y_pred_binary = (y_pred > 0.5)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
['10.45%' '10.36%' '41.20%' ... '50.21%' '3.47%' '19.62%']


#### Avaliando o modelo

In [16]:
# Gerando a matriz de confusão
# (compara as previsões com os valores reais para avaliar o desempenho do modelo)
cm = confusion_matrix(y_test, y_pred_binary)
print(cm)

[[1566   41]
 [ 274  119]]


In [17]:
# Transformando a matriz de confusão em um DataFrame do pandas
# (facilita a visualização e permite formatar melhor, se necessário)
cm = pd.DataFrame(cm)
cm

,0,1
0,1566,41
1,274,119


In [18]:
# Calculando manualmente a acurácia usando a matriz de confusão

tamanho_do_teste = y_test.shape[0]  # Total de amostras no conjunto de teste

verdadeiros_positivos = cm.values[0][0]  # Elementos corretamente classificados como 0 (classe negativa)
falsos_positivos = cm.values[1][1]       # Elementos corretamente classificados como 1 (classe positiva)

# Acurácia é a soma dos acertos (verdadeiros positivos + falsos positivos) dividido pelo total de amostras
acuracia = (verdadeiros_positivos + falsos_positivos) / tamanho_do_teste

# Exibindo a acurácia final da Rede Neural
print(f"Acurácia da RNA foi de {round(acuracia, 2) * 100}%")

Acurácia da RNA foi de 84.0%


# Salvando o modelo

In [19]:
folder = 'rna/'  # Nome da pasta onde o modelo será salvo

# Verifica se a pasta existe; se não, cria a pasta
if not os.path.exists(folder):
  os.makedirs(folder)

# Salva o modelo treinado no caminho especificado, com o nome 'rede_neural_one.h5'
classifier.save(os.path.join(folder, 'rede_neural_one.h5'))

# Importando o modelo para uso

In [21]:
# Carregando o modelo salvo
model = load_model(os.path.join(folder, 'rede_neural_one.h5'))

# Fazendo previsões
y_pred = model.predict(X_test)

# Aplicando o limiar de 0.7 para definir as classes
y_pred = (y_pred > 0.7)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [22]:
# Gerando a matriz de confusão com as novas previsões (threshold de 0.7)
cm = confusion_matrix(y_test, y_pred)

# Transformando a matriz em um DataFrame para melhor visualização
cm = pd.DataFrame(cm)

# Exibindo a matriz de confusão
cm

,0,1
0,1600,7
1,345,48
